# Welcome to the Synergy Roundtable
created by Stephan Koch & Dirk Derichsweiler - 01/11/2018<br>
do not hesitate to contact us: stephan.koch@hpe.com or dirk.derichsweiler@hpe.com



jupyter Notebook can be found: https://github.com/dderichswei/synergy


#### requirements
- Anaconda (Python) recommended (for the Jupyter installation and pip)
- Jupyter-Powershell (https://github.com/vors/jupyter-powershell)


#### additional information

On your HPE OneView appliance, or online<br>
https://10.0.20.50/help/cic-rest/en/content/index.html#home.html<br>
https://10.0.20.50/api-docs/current/ <br>
http://www.hpe.com/info/oneview/docs <br>
https://developer.hpe.com/

#### Powershell (POSH) specific
https://github.com/HewlettPackard/POSH-HPOneView<br>



### Installation of HP OneView Module
only required, if not installed yet.

In [ ]:
Start-Process powershell -ArgumentList "Install-Module HPOneView.410 -AllowClobber -Verb runAs"
Start-Process powershell -ArgumentList "Set-ExecutionPolicy RemoteSigned -Scope CurrentUser"


### Login

<img src="Pictures/Login.gif" height="500" width="500" align="left">

#### import the PowerShell module:

In [1]:
Import-Module -name hponeview.410 

#### login:

In [2]:
Connect-HPOVMgmt -Hostname 10.0.20.50 -AuthLoginDomain local -UserName python -Password python123!


ConnectionID Name       UserName AuthLoginDomain Default
------------ ----       -------- --------------- -------
1            10.0.20.50 python   local           True   




#### ADVANCED: which functions are available?

In [ ]:
Get-Command -Module HPOneView.410 

### show existing networks

<img src="Pictures/show_network.gif" height="500" width="500" align="left">
<br>

In [ ]:
Get-HPOVNetwork

### create network

<img src="Pictures/create_network.gif" height="500" width="500" align="left">

In [ ]:
New-HPOVNetwork -Name "Roundtable - Test Ethernet Network" -VlanId 200 -Type Ethernet -VLANType Tagged -Purpose General -SmartLink $False -PrivateNetwork $False  

### create bulk network
How to setup multiple networks at once.

In [ ]:
$net = @(1,2,3,4,5,7,100,101,102,103,104,105,106,107,108,109,200)  
foreach ($i in $net) { New-HPOVNetwork -Name "Bulk_$i" -VlanId $i -Type Ethernet -VLANType Tagged } 

### delete bulk network
as it´s not needed for the demo

In [ ]:
Get-HPOVNetwork -name "bulk*" | Remove-HPOVNetwork -Confirm:$false

### show configured/existing storage (systems and pools)

<img src="Pictures/show_storage.gif" height="500" width="500" align="left">

In [ ]:
Get-HPOVStorageSystem


In [ ]:
Get-HPOVStoragePool  | select Name, URI 

### show volume templates

In [ ]:
Get-HPOVStorageVolumeTemplate

### create volume template

<img src="Pictures/create_volume_template.gif" height="500" width="500" align="left">

In [ ]:
$StorPool=Get-HPOVStoragePool -Name "FC_r5" -StorageSystem 3par20k 
New-HPOVStorageVolumeTemplate -Name "Roundtable Volume Template" -StoragePool $StorPool -Capacity 10240 

### show Enclosure / Server Hardware / Bay
<img src="Pictures/show_server_hw.gif" height="500" width="500" align="left">

In [ ]:
Get-HPOVEnclosureGroup 

In [ ]:
Get-HPOVServerHardwareType

In [ ]:
Get-HPOVServer | select Name, URI

#### show Serverprofile

In [ ]:
Get-HPOVServerProfileTemplate|select Name, URI 

In [ ]:
Get-HPOVServerProfile|select Name, URI

### create server profile (takes 2-3 minutes)
<br>
<img src="Pictures/boot_synergy.gif" height="500" width="500" align="left">

In [4]:
### Get the first available server based on the template configuration
##$Server = Get-HPOVServer -InputObject $ServerProfileTemplate -NoProfile | Select -First 1

$Server=Get-HPOVServer -Name "CTC Synergy HE01, bay 8"

#Power off Server if on
Stop-HPOVServer -Server $Server -Force -Confirm:$false | Wait-HPOVTaskComplete

$Template=Get-HPOVServerProfileTemplate -Name "Roundtable - Python (DirkD)"
$OSDeploymentPlanName="RHEL 7.3 - customizable"

# Get the OS Deployment Plan
$OSDeploymentPlan = Get-HPOVOSDeploymentPlan -Name $OSDeploymentPlanName -ErrorAction Stop

# Get the associated deployment plan attributes
##$OSDeploymentAttributes = Get-HPOVOSDeploymentPlanAttribute -InputObject $OSDeploymentPlan -ErrorAction Stop
$OSDeploymentAttributes = Get-HPOVOSDeploymentPlanAttribute -InputObject $Template

($OSDeploymentAttributes | Where-Object name -eq "NewUser").value = "dirk"
($OSDeploymentAttributes | Where-Object name -eq "NewUserPassword").value = "HalloRoundTable!"

# Set root password, in clear text here, as OneView does not support secure string data types
#($OSDeploymentAttributes | Where-Object name -eq "NewRootPassword").value = [Runtime.InteropServices.Marshal]::PtrToStringAuto([Runtime.InteropServices.Marshal]::SecureStringToBSTR($Password))
($OSDeploymentAttributes | Where-Object name -eq "NewRootPassword").value = "Passw0rd"

# Set Hostname
#($OSDeploymentAttributes | Where-Object name -eq "Hostname").value = $ServerProfileName
($OSDeploymentAttributes | Where-Object name -eq "Hostname").value = "anyhostname"


$params = @{
        AssignmentType        = "Server";
        Description           = "HPE Synergy 480  Server ";
        Name                  = "Roundtable - API Demo Server (DirkD)2";
        Server                = $Server;
        ServerProfileTemplate = $Template;
        OSDeploymentAttributes = $OSDeploymentAttributes;
        OSDeploymentPlan = $OSDeploymentPlan
}

New-HPOVServerProfile @params | Wait-HPOVTaskComplete


#Power on Server
Start-HPOVServer -Server $Server -Confirm:$false  



#Power off Server if on
Stop-HPOVServer -Server $Server -Force -Confirm:$false | Wait-HPOVTaskComplete
